In [ ]:
! pip install kaggle
! mkdir ~/.kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
! cp /content/kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle competitions download -c tabular-playground-series-feb-2022

 98% 262M/267M [00:02<00:00, 126MB/s]
100% 267M/267M [00:02<00:00, 96.9MB/s]


In [ ]:
! unzip -f /content/tabular-playground-series-feb-2022.zip

Archive:  /content/tabular-playground-series-feb-2022.zip


In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split 
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from sklearn import preprocessing

import matplotlib.pyplot as plt

from sklearn.decomposition import PCA

In [ ]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
df_submission = pd.read_csv('sample_submission.csv')

FileNotFoundError: ignored

In [ ]:
df_train.head()

In [ ]:
df_train.shape

In [ ]:
df_train = df_train.drop(columns=['row_id'])

In [ ]:
df_test.head()

In [ ]:
test_data_row_ids = df_test['row_id']
df_test = df_test.drop(columns=['row_id'])

In [ ]:
df_train.isnull().sum()

In [ ]:
df_train.duplicated().sum()

In [ ]:
df_train.drop_duplicates(inplace=True)

In [ ]:
df_train.duplicated().sum()

In [ ]:
df_train.shape

In [ ]:
df_train['target'].unique()

In [ ]:
import seaborn as sns
sns.set(rc={'figure.figsize':(30, 10)})
sns.countplot(data = df_train, x = "target")

In [ ]:
df_train.groupby("target").describe()

In [ ]:
x = df_train.drop('target', axis=1)
x.head()

In [ ]:
y = df_train['target']
y.head()

In [ ]:
ss = preprocessing.StandardScaler()
x = ss.fit_transform(x)
x = pd.DataFrame(x)

In [ ]:
x.head()

In [ ]:
pca = PCA(n_components=20)
# pca = PCA(0.7)
x = pca.fit_transform(x)
x = pd.DataFrame(x)
# pca.n_components_

In [ ]:
x

In [ ]:
le = preprocessing.LabelEncoder()
y = le.fit_transform(y)

In [ ]:
y = y.reshape(-1, 1)

In [ ]:
np.unique(y, return_counts=True)

In [ ]:
df_tmp = x.copy()
df_tmp["target"] = y

In [ ]:
x

In [ ]:
y

In [ ]:
df_tmp.groupby("target").describe()

In [ ]:
corr = df_tmp.corr() 

plt.figure(figsize=(10,10))
sns.heatmap(corr,cbar=True,square=True,fmt='.2f',annot=True,annot_kws={'size':8}, cmap='Reds')

In [ ]:
avg = df_train.groupby(['target']).mean() # getting the average contributuion for each target
# avg.head()
avg.transpose().plot(kind='line',figsize=(25, 10))
plt.title('Contribution to the target')
plt.ylabel('Average contribution')
plt.xlabel('ATGC combination')
plt.show()

In [ ]:
avg = df_tmp.groupby(['target']).mean() # getting the average contributuion for each target
# avg.head()
avg.transpose().plot(kind='line',figsize=(25, 10))
plt.title('Contribution to the target')
plt.ylabel('Average contribution')
plt.xlabel('ATGC combination')
plt.show()

In [ ]:
x

SVM

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)

In [ ]:
X_train

In [ ]:
X_test

In [ ]:
Y_train

In [ ]:
Y_test

In [ ]:
from sklearn.svm import LinearSVC
support_vector_classifier1 = LinearSVC(C=1)
support_vector_classifier1.fit(X_train, Y_train)

Y_pred_train_svc1 = support_vector_classifier1.predict(X_train)
Y_pred_test_svc1 = support_vector_classifier1.predict(X_test)

In [ ]:
def print_scores(real, predicted):
    accuracy = accuracy_score(real, predicted)
    precision = precision_score(real, predicted, average=None)
    recall = recall_score(real, predicted, average=None)
    f1 = f1_score(real, predicted, average=None)

    print("Accuracy Score:  ", accuracy)
    print('Precision Score: ', precision)
    print('Recall Score:    ', recall)
    print('F-1 Score: ', f1)

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1_score,
    }

In [ ]:
svc1_train_scores = print_scores(Y_train, Y_pred_train_svc1)

In [ ]:
svc1_test_scores = print_scores(Y_test, Y_pred_test_svc1)

In [ ]:
df_test = ss.fit_transform(df_test)
df_test = pca.transform(df_test)
df_test = pd.DataFrame(df_test)

In [ ]:
df_test = pd.DataFrame(df_test)

In [ ]:
df_test

In [ ]:
y_pred_df_test = support_vector_classifier1.predict(df_test)
y_pred_df_test = le.inverse_transform(y_pred_df_test)

df_lsvm_submission = pd.DataFrame(
    {
        'row_id': test_data_row_ids,
        'target': y_pred_df_test
    }
)
df_lsvm_submission

df_lsvm_submission.to_csv('group2_model1_result.csv', index = False, encoding='utf-8')

In [ ]:
support_vector_classifier2 = SVC(kernel='poly', C=0.2, gamma='auto', probability=True)
support_vector_classifier2.fit(X_train, Y_train)

Y_pred_train_svc2 = support_vector_classifier2.predict(X_train)
Y_pred_test_svc2 = support_vector_classifier2.predict(X_test)

In [ ]:
svc2_train_scores = print_scores(Y_train, Y_pred_train_svc2)


In [ ]:
svc2_train_scores = print_scores(Y_test, Y_pred_test_svc2)


In [ ]:
y_pred_df_test = support_vector_classifier2.predict(df_test)
y_pred_df_test = le.inverse_transform(y_pred_df_test)

df_support_vector_classifier2_submission = pd.DataFrame(
    {
        'row_id': test_data_row_ids,
        'target': y_pred_df_test
    }
)
df_support_vector_classifier2_submission

df_support_vector_classifier2_submission.to_csv('group2_model2_result.csv', index=False, encoding='utf-8')

**Logistic Regression**

1.  Logistic Regression (From Description)





In [ ]:
logistic_regression1 = LogisticRegression(C=1, solver='liblinear')
logistic_regression1.fit(X_train, Y_train)
y_pred_train_log1 = logistic_regression1.predict(X_train)
y_pred_test_log1 = logistic_regression1.predict(X_test)


In [ ]:
logistic_regression1_train_scores = print_scores(Y_train, y_pred_train_log1)


In [ ]:
logistic_regression1_train_scores = print_scores(Y_test, y_pred_test_log1)


In [ ]:
y_pred_df_test = logistic_regression1.predict(df_test)
y_pred_df_test = le.inverse_transform(y_pred_df_test)

df_logistic_regression1_submission = pd.DataFrame(
    {
        'row_id': test_data_row_ids,
        'target': y_pred_df_test
    }
)
df_logistic_regression1_submission

df_logistic_regression1_submission.to_csv('group2_model3_result.csv', index = False, encoding='utf-8')

2. Logistic Regression (Applied)

In [ ]:
logistic_regression2 = LogisticRegression(multi_class = "multinomial", C=0.5, solver='saga')
logistic_regression2.fit(X_train, Y_train)
y_pred_train_log2 = logistic_regression2.predict(X_train)
y_pred_test_log2 = logistic_regression2.predict(X_test)

In [ ]:
logistic_regression2_train_scores = print_scores(Y_train, y_pred_train_log2)


In [ ]:
logistic_regression2_train_scores = print_scores(Y_test, y_pred_test_log2)


In [ ]:
y_pred_df_test = logistic_regression2.predict(df_test)
y_pred_df_test = le.inverse_transform(y_pred_df_test)

df_logistic_regression_submission = pd.DataFrame(
    {
        'row_id': test_data_row_ids,
        'target': y_pred_df_test
    }
)

print(df_logistic_regression_submission)

df_logistic_regression_submission.to_csv('group2_model4_result.csv', index = False, encoding='utf-8')

## ROC and AUC

In [ ]:
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score

pred_prob1 = logistic_regression1.predict_proba(X_test)
pred_prob2 = logistic_regression2.predict_proba(X_test)
# pred_prob3 = support_vector_classifier1.predict_proba(X_test)
pred_prob4 = support_vector_classifier2.predict_proba(X_test)


logistic1_fpr, logistic1_tpr, threshold = roc_curve(Y_test, pred_prob1[:,1], pos_label=1)
auc_log1 = auc(logistic1_fpr, logistic1_tpr)

logistic2_fpr, logistic2_tpr, threshold = roc_curve(Y_test, pred_prob2[:,1], pos_label=1)
auc_log2 = auc(logistic2_fpr, logistic2_tpr)

svc1_fpr, svc1_tpr, threshold = roc_curve(Y_test, Y_pred_test_svc1, pos_label=1)
auc_svc1 = auc(svc1_fpr, svc1_tpr)

svc2_fpr, svc2_tpr, threshold = roc_curve(Y_test, pred_prob2[:,1], pos_label=1)
auc_svc2 = auc(svc2_fpr, svc2_tpr)


plt.figure(figsize=(5, 5), dpi=100)
plt.plot(logistic1_fpr, logistic1_tpr, linestyle='-', label='Logistic 1 (auc = %0.3f)' % auc_log1)
plt.plot(logistic2_fpr, logistic2_tpr, linestyle='-', label='Logistic 2 (auc = %0.3f)' % auc_log2)
plt.plot(svc1_fpr, svc1_tpr, linestyle='-', label='SVC 1 (auc = %0.3f)' % auc_svc1)
plt.plot(svc2_fpr, svc2_tpr, linestyle='-', label='SVC 2 (auc = %0.3f)' % auc_svc2)

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')

plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score

pred_prob1 = logistic_regression1.predict_proba(X_test)

logistic1_fpr, logistic1_tpr, threshold = roc_curve(Y_test, pred_prob1[:,1], pos_label=1)
auc_log1 = auc(logistic1_fpr, logistic1_tpr)

plt.figure(figsize=(5, 5), dpi=100)
plt.plot(logistic1_fpr, logistic1_tpr, linestyle='-', label='Logistic 1 (auc = %0.3f)' % auc_log1)

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')

plt.legend()
plt.show()